In [2]:
import os
import sys
import librosa
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout, Flatten, Embedding
import pickle
from copy import deepcopy
import numpy as np

2024-06-17 00:47:06.911842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 00:47:08.471295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Getting Data from The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS) : https://zenodo.org/records/1188976#.XA1fK2hKhPb
f2 = open('model/feature.pkl','rb')
feature_all = pickle.load(f2)
f3 = open('model/label.pkl','rb')
labels = pickle.load(f3)
y = deepcopy(labels)
for i in range(len(y)):
    y[i] = int(y[i])
labels

['08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',
 '08',

Pre-processing and train-test split

In [6]:
n_labels = len(y)
n_unique_labels = len(np.unique(y))
one_hot_encode = np.zeros((n_labels,n_unique_labels))
f = np.arange(n_labels)
for i in range(len(f)):
    one_hot_encode[f[i],y[i]-1]=1


X_train,X_test,y_train,y_test = train_test_split(feature_all,one_hot_encode,test_size = 0.3,random_state=20)

Testing with model 1

In [13]:
model = Sequential()

model.add(Dense(X_train.shape[1],input_dim =X_train.shape[1],kernel_initializer='normal',activation ='relu'))

model.add(Dense(400, kernel_initializer='normal',activation ='relu'))

model.add(Dropout(0.2))

model.add(Dense(200,kernel_initializer='normal',activation ='relu'))

model.add(Dropout(0.2))

model.add(Dense(100,kernel_initializer='normal',activation ='relu'))

model.add(Dropout(0.2))

model.add(Dense(y_train.shape[1],kernel_initializer='normal',activation ='softmax'))

model.compile(loss = 'categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

model.fit(X_train,y_train, epochs=200,batch_size = 5,verbose=1)

model.evaluate(X_test,y_test)

/home/debo121/ml_project_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200


I0000 00:00:1718382673.522424    7682 service.cc:145] XLA service 0x7f8d80005460 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718382673.522481    7682 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
2024-06-14 17:31:13.607431: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-14 17:31:13.918197: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 52/202 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1250 - loss: 3.0882

I0000 00:00:1718382679.562284    7682 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

I0000 00:00:1718382801.302819   13981 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1718382801.938769   13985 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 20 bytes spill stores, 20 bytes spill loads



 1/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.2812 - loss: 1.8997

I0000 00:00:1718382804.865941   14105 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 4 bytes spill stores, 4 bytes spill loads



14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 223ms/step - accuracy: 0.2675 - loss: 1.9547


[1.9686882495880127, 0.2638888955116272]

Saving model

In [15]:
mlp_model = model.to_json()
with open('mlp_model_relu_adadelta.json','w') as j:
    j.write(mlp_model)
model.save_weights("mlp_relu_adadelta_model.weights.h5")

y_pred_model1 = model.predict(X_test)
y2 = np.argmax(y_pred_model1,axis=1)
y_test2 = np.argmax(y_test , axis = 1)

14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step


Trying to measure accuracy by another method

In [17]:
count = 0
for i in range(y2.shape[0]):
    if y2[i] == y_test2[i]:
        count+=1

print('Accuracy for model 1 : ' + str((count / y2.shape[0]) * 100))

Accuracy for model 1 : 26.38888888888889
